In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import util
%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
possibleConvFilters = util.createRangeFromMidpoint(38,38)

possibleConvAndPoolLayers = util.createRangeFromMidpoint(5,5)

possibleNumberOfFCLayers = util.createRangeFromMidpoint(22,17)
possibleNumberOfNeuronsPerFCLayer = util.createRangeFromMidpoint(9,9)

possibleNumberOfEpochs = util.createRangeFromMidpoint(40,28)

lastBestDropoutRate = 0.710042	
lastBestAdamLearningRate = 0.000350	
lastBestL2 = 71.708797	

trial = 20

In [3]:
train = util.loadData("training")
dev,test = util.loadData("testing",.5)

choose = np.random.choice

n_convFilters = []
n_convAndPoolLayers = []
n_FCLayers = []
n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

lastTrialForNotebook = trial + 10

while trial < lastTrialForNotebook:
    convFilters = choose(possibleConvFilters)

    convAndPoolLayers = choose(possibleConvAndPoolLayers)
    convAndPoolLayers = min([convAndPoolLayers,6])

    numberOfFCLayers = choose(possibleNumberOfFCLayers)
    numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

    numberOfEpochs = choose(possibleNumberOfEpochs)
    
    dropoutRate = (np.random.rand() + lastBestDropoutRate)/2
    adamLearningRate = util.generateAdamLearningRate(lastBestAdamLearningRate)
    L2Rate = util.generateL2(lastBestL2)
    

    model = util.createModel(convFilters, convAndPoolLayers,numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)

    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        print()
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        n_convFilters.append(convFilters)
        n_convAndPoolLayers.append(convAndPoolLayers)
        n_FCLayers.append(numberOfFCLayers)
        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = util.createModelParametersDF([convFilters],[convAndPoolLayers],
                                              [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        
modelParametersDF = util.createModelParametersDF(n_convFilters,n_convAndPoolLayers,
                                            n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                            dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
display(modelParametersDF.sort_values(by='trainScore', ascending=False))
    



In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-06-17 06:05:58.348653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-17 06:05:58.348811: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-17 06:05:59.040757: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 12s 156ms/step - loss: 0.3507 - f1_score: 0.6491
devScore


2024-06-17 06:23:48.807080: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-06-17 06:23:48.807235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 2s 113ms/step - loss: 0.3657 - f1_score: 0.6277
concluding trial  20

trainScore
75/75 [==============================] - 10s 132ms/step - loss: 0.3512 - f1_score: 0.6489
devScore
10/10 [==============================] - 1s 87ms/step - loss: 0.3696 - f1_score: 0.6244
concluding trial  21

trainScore
75/75 [==============================] - 17s 221ms/step - loss: 0.3537 - f1_score: 0.6479
devScore
10/10 [==============================] - 3s 167ms/step - loss: 0.3681 - f1_score: 0.6384
concluding trial  22

trainScore
75/75 [==============================] - 9s 110ms/step - loss: 0.3518 - f1_score: 0.6487
devScore
10/10 [==============================] - 1s 68ms/step - loss: 0.3748 - f1_score: 0.6196
concluding trial  23

trainScore
75/75 [==============================] - 16s 202ms/step - loss: 0.3501 - f1_score: 0.6506
devScore
10/10 [==============================] - 2s 153ms/step - loss: 0.3684 - f1_score: 0.6320
concluding trial  24

trainSco

,n_convFilters,n_convAndPoolLayers,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
4,50,3,20,5,35,0.519754,0.000335,36.507546,0.650586,0.631977
5,38,5,27,10,34,0.413904,0.001648,79.723962,0.649865,0.645058
6,27,6,24,8,42,0.742742,0.000319,149.665029,0.649643,0.638469
7,53,3,27,8,38,0.397359,0.000253,36.003587,0.649370,0.637643
0,37,6,21,11,32,0.375466,0.000484,35.866061,0.649075,0.627677
1,40,3,14,4,34,0.516650,0.003100,35.872623,0.648907,0.624442
9,37,4,18,11,39,0.731762,0.000799,36.019746,0.648853,0.645991
3,32,3,16,4,42,0.788332,0.004680,36.715225,0.648737,0.619625
8,48,5,18,12,28,0.492099,0.000631,39.805215,0.648512,0.650836
2,53,3,17,6,43,0.789814,0.000236,51.183713,0.647862,0.638418
